In [43]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 
from pymystem3 import Mystem
from nltk.corpus import stopwords
morph = MorphAnalyzer()
mystem = Mystem()
import spacy

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [2]:
def remove_tags(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\t', '', text)
    text = re.sub(r'\d+', '', text)
    return text

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if word.isalnum()]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [4]:
habr_texts = [clean(word_tokenize(remove_tags(text.lower()))) for text in open('habr_texts.txt',encoding="utf-8")]

попробуем сравнить ТМ на словах и на нграммах

In [78]:
#немного расширим список стоп-слов
stop_words = stopwords.words('russian')
stop_words.extend(['get', 'lt', 't', 'gt', 'set', 'urn', 'host', 'where'])

In [79]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

In [80]:
ns_habr_texts = remove_stopwords(habr_texts)

In [44]:
#для n-grams
ph = gensim.models.Phrases(habr_texts, scoring='default', threshold=0.4, 
                           common_terms=set(stopwords.words('russian','english'))) # можно указать слова, которые 
                                                                          #не будут учитываться
p = gensim.models.phrases.Phraser(ph)
ngrammed_habr_texts = p[habr_texts]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [85]:
dictinary = gensim.corpora.Dictionary(ns_habr_texts)

In [86]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [87]:
print(dictinary) #для слов

Dictionary(7092 unique tokens: ['a', 'address', 'api', 'architecture', 'async']...)


In [69]:
print(dictinary) #для нграмм

Dictionary(8625 unique tokens: ['a', 'address', 'api', 'architecture', 'azure']...)


Преобразуем наши тексты в мешки слов. 

In [88]:
corpus = [dictinary.doc2bow(text) for text in ns_habr_texts]

In [89]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=1) # если поддерживается многопоточность
#lsi = gensim.models.LdaModel(corpus, 200, id2word=dictinary, passes=5)

Посмотрим на топики.

In [90]:
lda.print_topics() #для слов

[(37,
  '0.004*"сервер" + 0.003*"x" + 0.003*"язык" + 0.003*"запрос" + 0.002*"user" + 0.002*"if" + 0.002*"строка" + 0.002*"name" + 0.002*"технология" + 0.002*"клиент"'),
 (96,
  '0.003*"if" + 0.003*"файл" + 0.003*"a" + 0.003*"amp" + 0.002*"материал" + 0.002*"сервер" + 0.002*"технология" + 0.002*"база" + 0.002*"элемент" + 0.002*"управление"'),
 (1,
  '0.010*"игра" + 0.003*"клиент" + 0.003*"игрок" + 0.003*"технология" + 0.003*"точка" + 0.003*"устройство" + 0.002*"сеть" + 0.002*"язык" + 0.002*"объект" + 0.002*"машина"'),
 (28,
  '0.007*"файл" + 0.005*"сайт" + 0.004*"сеть" + 0.004*"доступ" + 0.003*"amp" + 0.003*"инструмент" + 0.003*"запись" + 0.003*"участник" + 0.003*"метод" + 0.003*"программа"'),
 (83,
  '0.005*"клиент" + 0.003*"x" + 0.003*"список" + 0.003*"a" + 0.003*"px" + 0.003*"сервер" + 0.003*"точка" + 0.003*"устройство" + 0.002*"инструмент" + 0.002*"for"'),
 (97,
  '0.005*"клиент" + 0.004*"ms" + 0.004*"устройство" + 0.003*"for" + 0.003*"сервер" + 0.003*"значение" + 0.003*"b" + 0.003*

In [50]:
lda.print_topics() #для нграмм

[(61,
  '0.003*"игра" + 0.003*"файл" + 0.003*"команда" + 0.002*"разработчик" + 0.002*"приложение" + 0.002*"gt" + 0.002*"потому" + 0.002*"код" + 0.002*"тут" + 0.002*"список"'),
 (44,
  '0.004*"сайт" + 0.004*"тип" + 0.003*"сервер" + 0.003*"приложение" + 0.002*"команда" + 0.002*"метод" + 0.002*"значение" + 0.002*"тест" + 0.002*"запрос" + 0.002*"функция"'),
 (89,
  '0.004*"функция" + 0.003*"сайт" + 0.003*"код" + 0.002*"игра" + 0.002*"приложение" + 0.002*"устройство" + 0.002*"потому" + 0.002*"метод" + 0.002*"файл" + 0.002*"ошибка"'),
 (36,
  '0.005*"устройство" + 0.003*"команда" + 0.002*"ты" + 0.002*"сеть" + 0.002*"разработчик" + 0.002*"про" + 0.002*"задача" + 0.002*"приложение" + 0.002*"потому" + 0.002*"да"'),
 (79,
  '0.002*"сервер" + 0.002*"слово" + 0.002*"сеть" + 0.002*"надо" + 0.002*"ошибка" + 0.002*"сайт" + 0.002*"машина" + 0.002*"gt_lt" + 0.002*"код" + 0.002*"сервис"'),
 (24,
  '0.003*"технология" + 0.003*"устройство" + 0.003*"число" + 0.002*"программа" + 0.002*"карта" + 0.002*"датчи

Ещё есть штука для визуализации.

In [51]:
pyLDAvis.enable_notebook()

In [26]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary,sort_topics=True)

C:\Users\natus\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Можно посмотреть метрики.

In [91]:
import numpy as np

In [52]:
?lda.log_perplexity

In [53]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-68.81212930489701

In [92]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-51.95732477361044

In [93]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=ns_habr_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [94]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [95]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ns_habr_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [58]:
coherence_model_lda.get_coherence() #для нграмм

0.3511312700366961

In [96]:
coherence_model_lda.get_coherence() #для слов

0.3754757004263717

### Разложение матриц в sklearn

In [97]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [98]:
stexts = [' '.join(text) for text in ns_habr_texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [99]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

Разложим её.

In [100]:
model = NMF(n_components=30)

In [101]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [40]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [102]:
get_nmf_topics(model, 10)#для слов

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,продукт,public,игра,ия,файл,сервер,устройство,космический,сертификат,android,...,бот,объект,язык,windows,книга,блокчейн,виртуальный,российский,звук,товар
1,сотрудник,string,игрок,обучение,папка,сеть,камера,спутник,ключ,google,...,сообщение,значение,программирование,microsoft,часы,технология,резервный,страна,сигнал,скидка
2,клиент,new,игровой,нейросеть,скрипт,клиент,смартфон,орбита,пароль,мобильный,...,telegram,элемент,php,linux,профессиональный,транзакция,машина,россия,наушник,цена
3,бизнес,return,играть,робот,sudo,услуга,датчик,марс,домен,ios,...,телеграм,метод,программист,visual,порекомендовать,криптовалюта,вм,налог,частота,магазин
4,программист,var,персонаж,интеллект,пакет,сервис,аккумулятор,аппарат,сервер,app,...,чат,класс,java,azure,предпочитать,финансовый,veeam,закон,усилитель,распродажа
5,заказчик,private,steam,искусственный,директория,облачный,телефон,земля,письмо,устройство,...,канал,алгоритм,лекция,studio,путь,ibm,бэкап,рынок,звуковой,покупатель
6,опыт,class,vr,сеть,строка,инфраструктура,корпус,луна,сообщение,tv,...,мессенджер,точка,перевод,server,слушать,биткойна,копирование,рубль,музыка,покупка
7,менеджер,void,unity,нейронный,http,облако,печать,полёт,токен,play,...,api,изображение,программа,браузер,бумажный,платформа,восстановление,доход,искажение,пятница
8,деньга,класс,движок,машинный,настройка,трафик,видео,станция,подпись,библиотека,...,сервис,блок,локализация,обновление,друг,платёж,копия,налоговый,акустический,чёрный
9,идея,if,геймплей,алгоритм,модуль,оборудование,батарея,ракета,адрес,дайджест,...,bot,свойство,python,ос,пользоваться,платёжный,диск,миллион,диапазон,заказ


In [63]:
get_nmf_topics(model, 10) #для нграмм

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,изображение,gt_lt,игра,рынок,задача,сервер,смартфон,приложение,спутник,товар,...,сигнал,бот,php,лекция,сайт,камера,звук,ты,блокчейн,файл
1,алгоритм,lt_gt,игрок,россия,программист,диск,телефон,android,марс,скидка,...,частота,telegram,yii,курс,страница,видео,наушник,надо,транзакция,скрипт
2,текстура,gt,играть,страна,команда,вм,устройство,ios,орбита,магазин,...,устройство,телеграм,symfony,студент,домен,регистратор,усилитель,да,криптовалюта,пакет
3,функция,lt,игровой,российский,сотрудник,виртуальный_машина,ноутбук,api,земля,чёрный_пятница,...,плата,сообщение,laravel,видео,реклама,blackvue,музыка,потому,биткойна,папка
4,значение,lt_div,персонаж,бизнес,разработчик,облако,дисплей,мобильный,наса,цена,...,антенна,канал,язык,вступительный,клиент,движение,микрофон,про,bitcoin,команда
5,объект,компонент,геймплей,услуга,клиент,кластер,аккумулятор,устройство,ракета,распродажа,...,датчик,мессенджер,прислать,профессор,контент,съёмка,звучание,твой,технология,настройка
6,модель,div_gt,steam,закон,продукт,хост,экран,платформа,луна,покупатель,...,приёмник,api,go,биология,браузер,кадр,читать_слушать,ну,блок,репозиторий
7,элемент,android_android,уровень,налог,заказчик,бэкап,планшет,мобильный_приложение,аппарат,покупка,...,канал,чат,java,преподаватель,трафик,объектив,звуковой,потом,валюта,директория
8,нейросеть,lt_input,unity,доход,разработка,резервный_копирование,xiaomi,библиотека,станция,продавец,...,сеть,лига,redis,школа,сертификат,ivideon,колонка,думать,ethereum,the
9,точка,lt_li,vr,инвестор,ваш,инфраструктура,клавиатура,ваш_приложение,полёт,акция,...,ток,сервис,массив,программирование,https,видеонаблюдение,громкость,сказать,blockchain,nginx


Кажется, что NMF сработал лучше LDA в обоих случаях